In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
train_data = pd.read_csv('/data/ephemeral/home/data/train/train_ratings.csv')

# user X item 행렬 및 유저당 아이템 카운트
user_items = train_data.groupby('user')['item'].apply(list).reset_index(name='items')
user_items['item_count'] = user_items['items'].apply(len)

user_items

,user,items,item_count
0,11,"[4643, 170, 531, 616, 2140, 2722, 2313, 2688, ...",376
1,14,"[8961, 1396, 471, 2105, 1042, 1947, 1269, 2394...",180
2,18,"[1952, 1283, 3507, 4280, 51084, 593, 318, 356,...",77
3,25,"[261, 22, 2161, 3255, 372, 1093, 428, 175, 214...",91
4,31,"[260, 1196, 1210, 7153, 4993, 5952, 1270, 5855...",154
...,...,...,...
31355,138473,"[524, 3354, 1025, 6565, 69757, 2085, 32, 55282...",63
31356,138475,"[1639, 1673, 1148, 246, 2019, 1267, 1172, 1235...",124
31357,138486,"[2694, 1994, 2723, 441, 2288, 637, 2013, 2423,...",137
31358,138492,"[2115, 908, 58, 2700, 2599, 1500, 1358, 1288, ...",68


In [3]:
# 앙상블할 기존의 제출 데이터

# bert(sequence) predictions
output_bert = pd.read_csv('/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/seq2.csv')

# ease(static) predictions
output_ease = pd.read_csv('/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/ease.csv')

In [6]:
# static 모델의 예측값중 몇개를 사용할 것인지에 대한 함수
def calculate_static_limit(count):
    k = 10
    val = count / 100
    if val >= k:
        return k - 3
    else:
       return int((40 / (val + 40)) * k) - 1

# 각 유저마다(row) 앙상블 적용함수
def combine_ensemble(row, k=10):
    limit = int(row['limit_k'])
    
    # 1) BERT에서 가져올 것 (limit 개수만큼)
    bert_selected = row['top_k_bert'][:k - limit]
    
    # 2) EASE에서 가져올 것 (나머지 채우기)
    # 단순히 [:k-limit]만 가져오면, 중복 제거 후 개수가 모자랄 수 있음.
    # 그래서 BERT 전체 리스트를 순회하며 채워넣음.
    final_list = list(bert_selected) # 복사
    
    ease_candidates = row['top_k_ease']
    
    for item in ease_candidates:
        if len(final_list) >= k: # 10개가 찼으면 중단
            break
        if item not in final_list:
            final_list.append(item)

    return final_list

# 앙상블 적용후 제출 df 생성
def ensemble():
    k = 10
    top_k_output_bert = output_bert.groupby('user')['item'].apply(list).reset_index(name='top_k_bert')[['user','top_k_bert']]
    top_k_output_ease = output_ease.groupby('user')['item'].apply(list).reset_index(name='top_k_ease')[['user','top_k_ease']]
    
    merged_df = top_k_output_ease.merge(user_items, on='user', how='left')
    merged_df = top_k_output_bert.merge(merged_df, on='user', how='left')
    merged_df['limit_k'] = merged_df['item_count'].apply(calculate_static_limit)
    
    merged_df['ensemble_list'] = merged_df.apply(lambda row: combine_ensemble(row, k=10), axis=1)
    # print(merged_df)
    return merged_df[['user', 'ensemble_list']]

In [7]:
# 앙상블 실행
ensemble_data = ensemble()


## submission 제출파일 생성


# 1. 리스트를 행으로 펼치기 (explode 사용)
# ensemble_list 컬럼의 [item1, item2, ...] 리스트가 각각의 행으로 분리됩니다.
submission_df = ensemble_data.explode('ensemble_list')

# 2. 컬럼 이름 변경 (ensemble_list -> item)
submission_df = submission_df.rename(columns={'ensemble_list': 'item'})

# 3. 필요한 컬럼만 선택 ('user', 'item' 순서)
final_submission = submission_df[['user', 'item']]

# 4. CSV 파일로 저장
# index=False를 해야 불필요한 인덱스 번호가 저장되지 않습니다.
final_submission.to_csv('submission_58.csv', index=False)

# 결과 확인
print(final_submission.head())

   user   item
0    11  55820
0    11   5995
0    11   4370
0    11   4886
0    11  40815


In [1]:
# EASE와 SLIM 추천 결과 비교하여 유저별 겹치는 아이템 비율 계산
import pandas as pd
import numpy as np

# EASE와 SLIM 추천 데이터 로드
ease = pd.read_csv("/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/ease.csv")
slim = pd.read_csv("/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/src/models/SLIM/outputs/recommendations.csv")

# 유저별로 아이템 리스트로 변환
ease_items = ease.groupby('user')['item'].apply(list).reset_index(name='ease_items')
slim_items = slim.groupby('user')['item'].apply(list).reset_index(name='slim_items')

# 두 데이터프레임 병합
comparison_df = ease_items.merge(slim_items, on='user', how='inner')

# 유저별 겹치는 아이템 수와 비율 계산
def calculate_overlap(row):
    ease_set = set(row['ease_items'])
    slim_set = set(row['slim_items'])
    
    # 교집합 (겹치는 아이템)
    overlap = ease_set.intersection(slim_set)
    
    # 합집합 (전체 고유 아이템)
    union = ease_set.union(slim_set)
    
    # 겹치는 아이템 수
    overlap_count = len(overlap)
    
    # 전체 아이템 수 (합집합)
    total_count = len(union)
    
    # 겹치는 비율 (%)
    overlap_percentage = (overlap_count / total_count) * 100 if total_count > 0 else 0
    
    return {
        'overlap_count': overlap_count,
        'total_count': total_count,
        'overlap_percentage': overlap_percentage,
        'overlap_items': list(overlap)
    }

# 각 유저별로 겹치는 비율 계산
overlap_results = comparison_df.apply(calculate_overlap, axis=1, result_type='expand')

# 결과를 원본 데이터프레임에 추가
comparison_df = pd.concat([comparison_df, overlap_results], axis=1)

# 전체 유저의 평균 겹치는 비율
average_overlap = comparison_df['overlap_percentage'].mean()

print(f"전체 유저 평균 겹치는 아이템 비율: {average_overlap:.2f}%")
print(f"분석된 유저 수: {len(comparison_df)}")
print()

# 상위 10개 유저의 겹치는 비율 확인
print("상위 10개 유저의 겹치는 비율:")
print(comparison_df[['user', 'overlap_count', 'total_count', 'overlap_percentage']].head(10))

# 겹치는 비율 분포 확인
print(f"\n겹치는 비율 통계:")
print(comparison_df['overlap_percentage'].describe())

전체 유저 평균 겹치는 아이템 비율: 58.74%
분석된 유저 수: 31360

상위 10개 유저의 겹치는 비율:
   user  overlap_count  total_count  overlap_percentage
0    11              9           11           81.818182
1    14              6           14           42.857143
2    18              8           12           66.666667
3    25              7           13           53.846154
4    31              8           12           66.666667
5    35             10           10          100.000000
6    43              8           12           66.666667
7    50              9           11           81.818182
8    58              6           14           42.857143
9    60              7           13           53.846154

겹치는 비율 통계:
count    31360.000000
mean        58.742749
std         16.101193
min          0.000000
25%         42.857143
50%         53.846154
75%         66.666667
max        100.000000
Name: overlap_percentage, dtype: float64


In [ ]:
ease = pd.read_csv("/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/ease.csv")
slim = pd.read_csv("/data/ephemeral/home/jimin/pro-recsys-movierecommendation-recsys-04/src/models/SLIM/outputs/recommendations.csv")

In [2]:
train_df = pd.read_csv("/data/ephemeral/home/data/train/train_ratings.csv")

In [3]:
train_df['time'] = pd.to_datetime(train_df['time'], unit='s').dt.year
train_df

,user,item,time
0,11,4643,2009
1,11,170,2009
2,11,531,2009
3,11,616,2009
4,11,2140,2009
...,...,...,...
5154466,138493,44022,2009
5154467,138493,4958,2009
5154468,138493,68319,2009
5154469,138493,40819,2009


In [4]:
train_df.to_csv("/data/ephemeral/home/data/train/train_ratings_years.csv", index=False)

In [8]:
last_rows = train_df.groupby('user').last().reset_index()
last_rows['time'] = pd.to_datetime(last_rows['time'], unit='s').dt.year
last_rows

,user,item,time
0,11,4226,1970
1,14,468,1970
2,18,71033,1970
3,25,52319,1970
4,31,5621,1970
...,...,...,...
31355,138473,3578,1970
31356,138475,6611,1970
31357,138486,6502,1970
31358,138492,3210,1970


In [9]:
last_rows.to_csv("/data/ephemeral/home/data/train/train_ratings_years.csv", index=False)

In [5]:
years = pd.read_csv("/data/ephemeral/home/data/train/years.tsv", sep='\t')

In [6]:
years

,item,year
0,1348,1922
1,44587,1922
2,4768,1922
3,8235,1923
4,8609,1923
...,...,...
6794,114795,2014
6795,110771,2014
6796,112804,2014
6797,113378,2014
